In [5]:
# Import required libraries
import pandas as pd
import yfinance as yf
import time
import re
from datetime import datetime

def get_nyse_breadth_data(start_date, end_date, csv_file_path='NYSE.csv', export_path='nyse_breadth_data.csv'):
    """
    Retrieves NYSE ticker data, calculates daily advances/declines/neutral,
    and exports the data to a CSV file.
    
    Args:
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format
        csv_file_path (str): Path to the NYSE tickers CSV file
        export_path (str): Path for the output CSV file
    
    Returns:
        pd.DataFrame: DataFrame with daily advances, declines, and neutral counts
    """
    print(f"Running NYSE breadth analysis from {start_date} to {end_date}...")
    
    # Step 1: Retrieve NYSE tickers
    try:
        nyse_data = pd.read_csv(csv_file_path)
        tickers = nyse_data['Symbol'].tolist()
        
        # Clean tickers for yfinance
        clean_tickers = []
        for ticker in tickers:
            ticker_str = str(ticker)
            if not re.search(r'[\^/\.\-]', ticker_str):
                clean_tickers.append(ticker_str)
        
        print(f"Retrieved {len(clean_tickers)} clean NYSE ticker symbols")
    except Exception as e:
        print(f"Error retrieving NYSE tickers: {e}")
        return None
    
    # Step 2: Get daily price data in batches
    batch_size = 500
    all_data = pd.DataFrame()
    
    for i in range(0, len(clean_tickers), batch_size):
        batch_tickers = clean_tickers[i:i+batch_size]
        print(f"Downloading data for tickers {i+1} to {min(i+batch_size, len(clean_tickers))}...")
        
        try:
            batch_data = yf.download(batch_tickers, start=start_date, end=end_date, progress=False)
            
            if len(batch_tickers) > 1:
                batch_close = batch_data['Close']
            else:
                batch_close = batch_data['Close'].to_frame(name=batch_tickers[0])
            
            if all_data.empty:
                all_data = batch_close
            else:
                all_data = all_data.join(batch_close, how='outer')
            
            time.sleep(1)  # Avoid hitting API limits
            
        except Exception as e:
            print(f"Error downloading data for batch starting at index {i}: {e}")
    
    print(f"Downloaded daily price data with shape {all_data.shape}")
    
    # Step 3: Calculate daily advances, declines, and neutral
    daily_changes = all_data.pct_change()
    
    advances = (daily_changes > 0).sum(axis=1)
    declines = (daily_changes < 0).sum(axis=1)
    neutral = (daily_changes == 0).sum(axis=1)
    
    # Create DataFrame with results
    breadth_data = pd.DataFrame({
        'Advancers': advances,
        'Decliners': declines,
        'Neutral': neutral
    })
    
    # Drop the first row which has all zeros due to no percentage change calculation
    breadth_data = breadth_data.iloc[1:]
    # Step 4: Export to CSV
    try:
        breadth_data.to_csv(export_path)
        print(f"Successfully exported breadth data to {export_path}")
    except Exception as e:
        print(f"Error exporting data to CSV: {e}")
    
    return breadth_data

# Example usage:
breadth_data = get_nyse_breadth_data('2018-01-01', '2025-04-23', 'NYSE.csv', 'nyse_breadth_2023.csv')


Running NYSE breadth analysis from 2018-01-01 to 2025-04-23...
Retrieved 2356 clean NYSE ticker symbols



3 Failed downloads:
['EAI']: YFPricesMissingError('possibly delisted; no price data found  (1d 2018-01-01 -> 2025-04-23)')
['ECC           ', 'ETX           ']: YFPricesMissingError('possibly delisted; no price data found  (1d 2018-01-01 -> 2025-04-23) (Yahoo error = "No data found, symbol may be delisted")')



2 Failed downloads:
['SFB']: YFPricesMissingError('possibly delisted; no price data found  (1d 2018-01-01 -> 2025-04-23)')
['SAND          ']: YFPricesMissingError('possibly delisted; no price data found  (1d 2018-01-01 -> 2025-04-23) (Yahoo error = "No data found, symbol may be delisted")')


Downloaded daily price data with shape (1836, 2356)
Successfully exported breadth data to nyse_breadth_2023.csv


/var/folders/0l/68dwwf7n4k14jjgryv9tt5mw0000gn/T/ipykernel_27623/952589310.py:70: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  daily_changes = all_data.pct_change()
